In [7]:
%reload_ext autoreload
%autoreload 2

In [8]:
import os
import sys
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + 'C:\\Users\\rdpuser\\Documents\\Github\\parkit;'
sys.path.append('C:\\Users\\rdpuser\\Documents\\Github\\parkit')

In [9]:
import parkit as pk

pk.import_site('C:\\Users\\rdpuser\\Desktop\\test')

In [10]:
# Smoke Tests

In [11]:
smoke_tests = pk.bind_task('smoke_tests')

In [16]:
pk.pool().start()

In [19]:
traces = []
for i in range(16):
    traces.append(smoke_tests(1000))

In [20]:
pk.wait(*traces, lambda: all(trace.done for trace in traces))
assert all(trace.result for trace in traces)

In [21]:
pk.pool().stop()

In [ ]:
# File Tests

In [22]:
import numpy as np
import pandas as pd

In [23]:
rows = []
for i in range(10000):
    rows.append(dict(a = i, b = i))
df = pd.DataFrame(rows)

In [24]:
f = pk.File(mode = 'wb')

In [25]:
with f:
    df.to_feather(f)

In [26]:
f.mode = 'rb'
with f:
    assert df.equals(pd.read_feather(f))

In [27]:
a = np.zeros((100,100))

In [28]:
f = pk.File(mode = 'wb+')
with f:
    np.save(f, a)

In [29]:
f.mode = 'rb+'
with f:
    assert np.array_equal(a, np.load(f))